# ImageNet Dataset Exploration

This notebook aims to explore and prep the ImageNet(IN) dataset for training on YOLOv1 model. The model is built on the Tensorflow framework. The IN dataset can be downloaded from [here](http://image-net.org/download). Download and extract it. The path to the home directory(ILSVRC) should be stored in the following variable:

In [ ]:
absolute_ImageNet_path = "/home/aditya/Documents/public_data/ILSVRC"
classification_path = absolute_ImageNet_path + "/Data/CLS-LOC"
classification_training_path = classification_path + "/train/"
classification_validation_path = classification_path + "/val/"

Two things that this notbook tries to achieve
- Explore, visualize and learn how the data and the corresponding labels are structured.
- Try and device an efficient way to load data at runtime for training. For obvious reasons(Limited CPU), we cannot work with hdf5 file formats or Tensor records.

In [ ]:
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import scipy
from scipy import misc
import sys
import skimage
from skimage import transform
from skimage.viewer import ImageViewer
import h5py
import json
from random import shuffle
import tensorflow as tf
import xml.etree.ElementTree as ET

In [ ]:
main_folders = os.listdir(absolute_ImageNet_path)
classification_folders = os.listdir(classification_path)
train_folder = os.listdir(classification_path+"/train")
val_folder = os.listdir(classification_path+"/val")
test_folder = os.listdir(classification_path+"/test")
print(len(main_folders))
print(len(classification_folders), len(train_folder), len(val_folder), len(test_folder))
sorted_train_folder = sorted(train_folder)

reference_dictionary = {}

# code to convert file names to numbers for ease of data-processing
n = 0
for folder in sorted_train_folder:
    reference_dictionary[folder] = n
    n += 1
    
# with open('./ImageNet_dataset/class_coding.txt', 'w') as file:
#     file.write(json.dumps(reference_dictionary))

Let's visualise some data

In [ ]:
path = classification_path+"/train/"+train_folder[1]
images = os.listdir(path)

# code to display an image
im = Image.open(path+'/'+images[19])
#im.show()

# code to resize the image into 224x224
plt.imshow(skimage.transform.resize((np.asarray(im)), (224, 224)))

a = np.asarray(im)[None]
a

In [ ]:
# test script : DELETE
im.close()

Our main interest is with the classification folder under Data. Creating hd5 files of different sizes for train/val/test folders would be a good way to go about it. The main problem is to replicate randomness in the way it is stored. We dont want images of the same class to be saved in a sequence.

Let us create 130 h5 files, each file containing 10,000 images. The distribution has to be kept homogenous.

steps involved:
- check for different image modes and format in the dataset, convert to RGB if different
- resize the images to 224x224
- create separate homogenous lists of image and labelled data
- shuffle the lists to make them random and distributed for the model to be trained on
- create multiple h5 files to load dynamically while training

In [ ]:
# check whether all train folders have 1300 images
# print out anomolous folders and the number of images they contain
anomolous_folders = []
for folder in train_folder:
    if (len(os.listdir(classification_path+"/train/"+folder)) == 1300):
        continue
    else:
        print(folder)
        anomolous_folders.append(folder)
print(len(anomolous_folders))
len(os.listdir(classification_path+"/train/"+anomolous_folders[104]))

In [ ]:
# test script : DELETE
image_files = os.listdir(classification_training_path+train_folder[0])
image = Image.open(classification_training_path+train_folder[0]+'/'+image_files[2])
print(image.mode)
image.close()

Following cell checks all the images in the dataset for the stored format. The model we train needs to have a fixed dimension of images - 224x224x3. There are a few images in the dataset that are of different dimensions. Lets get to the bottom of that.

In [ ]:
# DON'T RUN THIS CELL
# code to check all the different image formats and dimensions.
image_mode_dict = {}
for folder in train_folder:
    temp_path = classification_training_path+folder
    image_files = os.listdir(temp_path)
    for image_file in image_files:
        try:
            image = Image.open(temp_path + '/' + image_file)
            if image.mode in image_mode_dict:
                image_mode_dict[image.mode] += 1
            else:
                image_mode_dict[image.mode] = 1
                
        finally:
            image.close()
print(image_mode_dict)

Following cell implements all the steps mentioned earlier.

In [ ]:
# test script: DELETE
h5file_path = "./ImageNet_dataset/training_folder/train_test_file.hdf5"

image_data = []
label_data = []

test_path = classification_training_path+train_folder[0]
image_files = os.listdir(test_path)
for image_file in image_files:
    # code to check the number of channels of each image loaded and resize appropriately
    image = Image.open(test_path+'/'+image_file)
    if image.mode == 'L':
        rgbimage = Image.new("RGB", image.size)
        rgbimage.paste(image)
        resized_image = skimage.transform.resize((np.asarray(rgbimage)), (224, 224))
        image_data.append(resized_image)
        label_data.append(reference_dictionary[folder])
        rgbimage.close()
    elif image.mode == 'RGB':
        resized_image = skimage.transform.resize((np.asarray(image)), (224, 224))
        image_data.append(resized_image)
        label_data.append(reference_dictionary[folder])
    elif image.mode == 'RGBA':
        image_array = np.asarray(image)
        resized_image = skimage.transform.resize(image_array[:, :, :3], (224, 224))
        image_data.append(resized_image)
        label_data.append(reference_dictionary[folder])
    image.close()
    
# initializing the h5 file
X_dataset_shape = (len(image_data), 224, 224, 3)
Y_dataset_shape = (len(image_data),)
h5file = h5py.File(h5file_path, mode = 'w')
h5file.create_dataset("X_train", X_dataset_shape, np.float32)
h5file.create_dataset("Y_train", Y_dataset_shape, np.uint16)

# shuffling the data
c = list(zip(image_data, label_data))
shuffle(c)
image_data, label_data = zip(*c)

#plt.imshow(image_data[0])
print(len(image_data))
print(len(label_data))

# storing the data in the h5 file
length_image_data = len(image_data)
h5file["X_train"][...] = image_data
h5file["Y_train"][...] = label_data

plt.imshow(h5file['X_train'][0])
h5file.close()

In [ ]:
# test script: DELETE
h5_path = "./ImageNet_dataset/training_folder/train_test_file.hdf5"
h5_file = h5py.File(h5_path, mode = 'r')
total_num_samples = h5_file["X_train"].shape
total_labels = h5_file["Y_train"].shape
print(total_num_samples, total_labels)

image = h5_file['X_train'][0]
plt.imshow(image)
print(image)
h5_file.close()

## Creation of Training h5 files

In [ ]:
# code to resize and store the images in a h5 file.
n = 0
count = 0
folders_skipped = 0
sample = 2
while folders_skipped != 1000:
        
    # code to initialize a h5 file
    h5file_path = "./ImageNet_dataset/training_folder/train"+str(count)+".hdf5"

    # code to initialize a global variable to store the image data
    image_data = []
    label_data = []
    
    # variable to keep a track of number of skipped folders
    folders_skipped = 0

    # code to recursively go through each folder and select 10 images to store
    for folder in train_folder:
        temp_path = classification_training_path+folder
        image_files = os.listdir(temp_path)
        if (n >= len(image_files)):
            folders_skipped += 1
            continue
        elif ((len(image_files) - 1) - n < sample):
            concerned_image_files = image_files[n:]
        else:
            concerned_image_files = image_files[n:n+sample]
        for image_file in concerned_image_files:

            # code to check the number of channels of each image loaded and resize appropriately
            image = Image.open(temp_path+'/'+image_file)
            if image.mode == 'L':
                rgbimage = Image.new("RGB", image.size)
                rgbimage.paste(image)
                resized_image = skimage.transform.resize((np.asarray(rgbimage)), (224, 224))
                image_data.append(resized_image)
                label_data.append(reference_dictionary[folder])
                rgbimage.close()
            elif image.mode == 'RGB':
                resized_image = skimage.transform.resize((np.asarray(image)), (224, 224))
                image_data.append(resized_image)
                label_data.append(reference_dictionary[folder])
            elif image.mode == 'RGBA':
                image_array = np.asarray(image)
                resized_image = skimage.transform.resize(image_array[:, :, :3], (224, 224))
                image_data.append(resized_image)
                label_data.append(reference_dictionary[folder])
            image.close()

    # increment n
    n += sample

    # initializing the h5 file
    X_dataset_shape = (len(image_data), 224, 224, 3)
    Y_dataset_shape = (len(image_data),)
    h5file = h5py.File(h5file_path, mode = 'w')
    h5file.create_dataset("X_train", X_dataset_shape, np.float16)
    h5file.create_dataset("Y_train", Y_dataset_shape, np.uint16)

    # shuffling the data
    c = list(zip(image_data, label_data))
    shuffle(c)
    image_data, label_data = zip(*c)
    
    print(len(image_data))
    print(len(label_data))

    # storing the data in the h5 file
    h5file["X_train"][...] = image_data
    h5file["Y_train"][...] = label_data
    h5file.close()
    count += 1

In [ ]:
print(image.mode)
image_array = np.asarray(image)
image_array.shape
print(n)
h5file.close()

The cell below checks whether the h5 files are accurate by displaying a random image and printing the corresponding label. We can match the label against the semantic names of individual class codes [here](https://github.com/Lasagne/Recipes/blob/master/examples/resnet50/imagenet_classes.txt).(don't forget to add 1 to the coding as it is 0 indexed).

In [ ]:
# code to check h5 files
h5_path = "./ImageNet_dataset/training_folder/train"+str(600)+".hdf5"


with h5py.File(h5_path, mode = 'r') as h5_file:
    total_num_samples = h5_file["X_train"].shape[0]
    total_labels = h5_file["Y_train"].shape[0]
    print(total_num_samples, total_labels)

    image = h5_file['X_train'][500]
    label = h5_file['Y_train'][500]
    # viewer = ImageViewer(image)
    # viewer.show()
    # print(image)
    for classes, num in reference_dictionary.items():
        if num == label:
            print(num)
    misc.imshow(image)

Lets experiment with the tf.one_hot to generate labels

In [ ]:
with h5py.File(h5_path, mode = 'r') as h5_file:
    Y_train = np.asarray(h5_file['Y_train'])
    print(Y_train.shape)
    print(np.asarray(tf.one_hot(Y_train, 1000)))

## Creation of Validation h5 files

In [ ]:
# helper function
def parse_validation_xml(xml_file):
    """
    absolute path of ImageNet validation xml file
    """
    result = ''
    tree = ET.parse(xml_file)
    root = tree.getroot()
    for child in root:
        if (child.tag == 'object'):
            for sub_child in child:
                if (sub_child.tag == 'name'):
                    result = sub_child.text
     
    return result

In [ ]:
try:
    # code to initialize a h5 file
    H5FILE_VAL_PATH = "/media/aditya/Seagate Backup Plus Drive/h5_FILES_Aditya/validation_folder/val.hdf5"
    ANN_PATH = '/home/aditya/Documents/public_data/ILSVRC/Annotations/CLS-LOC/val/' 



    # code to initialize a global variable to store the image data
    image_data = []
    label_data = []

    # image files
    image_files = os.listdir(classification_validation_path)
    length = len(image_files)

    # initializing the h5 file
    X_dataset_shape = (length, 224, 224, 3)
    Y_dataset_shape = (length,)
    h5file = h5py.File(H5FILE_VAL_PATH, mode = 'w')
    h5file.create_dataset("X_val", X_dataset_shape, np.float16)
    h5file.create_dataset("Y_val", Y_dataset_shape, np.uint16)

    for i in range(length):

        # code to retrieve/construct the validation annotation string
        image_class = image_files[i].split('.')
        xml_file = image_class[0]+'.xml'

        # code to check the number of channels of each image loaded and resize appropriately
        image = Image.open(classification_validation_path+image_files[i])
        if image.mode == 'L':
            rgbimage = Image.new("RGB", image.size)
            rgbimage.paste(image)
            resized_image = skimage.transform.resize((np.asarray(rgbimage)), (224, 224))

            # save the image and label data
            h5file["X_val"][i, ...] = resized_image[None]
            label_data.append(reference_dictionary[parse_validation_xml(ANN_PATH+xml_file)])
            rgbimage.close()
        elif image.mode == 'RGB':
            resized_image = skimage.transform.resize((np.asarray(image)), (224, 224))
            h5file["X_val"][i, ...] = resized_image[None]
            label_data.append(reference_dictionary[parse_validation_xml(ANN_PATH+xml_file)])
        elif image.mode == 'RGBA':
            image_array = np.asarray(image)
            resized_image = skimage.transform.resize(image_array[:, :, :3], (224, 224))
            h5file["X_val"][i, ...] = resized_image[None]
            label_data.append(reference_dictionary[parse_validation_xml(ANN_PATH+xml_file)])
        image.close()
        print(image_files[i])

    

    # storing the data in the h5 file
    h5file["Y_val"][...] = label_data
finally:
    image.close()
    h5file.close()

In [ ]:
image.close()
h5file.close()

In [ ]:
# exploring the validation annotations
ANN_PATH = '/home/aditya/Documents/public_data/ILSVRC/Annotations/CLS-LOC/val/'
xml_files = os.listdir(ANN_PATH)
print(xml_files)

In [ ]:
parse_validation_xml(ANN_PATH+xml_files[0])

That marks the end of that. The tests were good, and the ImageNet dataset is ready to be trained upon.